# Token API

In [ ]:
#| default_exp token

In [ ]:
#| export

from typing import Tuple
from dataclasses import dataclass
from sugar.helpers import normalize_address

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

@dataclass(frozen=True)
class Token:
    """Data class for Token
    based on: https://github.com/velodrome-finance/sugar/blob/v2/contracts/LpSugar.vy#L17
    """

    token_address: str
    symbol: str
    decimals: int
    listed: bool

    def value_from_bigint(self, value: float) -> float:
        return value / 10**self.decimals

    @classmethod
    def from_tuple(cls, t: Tuple) -> "Token":
        (token_address, symbol, decimals, _, listed) = t
        return Token(
            token_address=normalize_address(token_address),
            symbol=symbol,
            decimals=decimals,
            listed=listed,
        )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()